### Introdution to joins

In [1]:
import pandas as pd
import sqlalchemy
import sqlite3
import os

In [2]:
conn = sqlite3.connect('leaders/leaders.sqlite')
cur = conn.cursor()

In [3]:
def sql_query(query):
    c = cur.execute(query)
    df = pd.DataFrame(c.fetchall(), 
                      columns=[description[0] for description in c.description])
    return df

In [4]:
sql_query('select * from presidents')

,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi
1,Portugal,Europe,Marcelo Rebelo de Sousa
2,Haiti,North America,Jovenel Moise
3,Uruguay,South America,Jose Mujica
4,Liberia,Africa,Ellen Johnson Sirleaf
5,Chile,South America,Michelle Bachelet
6,Vietnam,Asia,Tran Dai Quang


In [5]:
sql_query('''SELECT p1.country, p1.continent,
                    prime_minister, president
            FROM prime_ministers AS p1
            INNER JOIN presidents AS p2
            ON p1.country=p2.country''')

,country,continent,prime_minister,president
0,Egypt,Africa,Sherif Ismail,Abdel Fattah el-Sisi
1,Portugal,Europe,Antonio Costa,Marcelo Rebelo de Sousa
2,Vietnam,Asia,Nguyen Xuan Phuc,Tran Dai Quang
3,Haiti,North America,Jack Guy Lafontant,Jovenel Moise


In [6]:
conn.close()

In [7]:
conn = sqlite3.connect('countries/countries.sqlite')
cur = conn.cursor()

In [10]:
sql_query('SELECT * FROM countries')

,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.176100,34.522800
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581,Nederland,Constitutional Monarchy,Amsterdam,4.890950,52.373800
2,ALB,Albania,Europe,Southern Europe,28748.0,1912,Shqiperia,Republic,Tirane,19.817200,41.331700
3,DZA,Algeria,Africa,Northern Africa,2381740.0,1962,Al-Jazair/Algerie,Republic,Algiers,3.050970,36.739700
4,ASM,American Samoa,Oceania,Polynesia,199.0,None,Amerika Samoa,US Territory,Pago Pago,-170.691000,-14.284600
5,AND,Andorra,Europe,Southern Europe,468.0,1278,Andorra,Parliamentary Coprincipality,Andorra la Vella,1.521800,42.507500
6,AGO,Angola,Africa,Central Africa,1246700.0,1975,Angola,Republic,Luanda,13.242000,-8.811550
7,ATG,Antigua and Barbuda,North America,Caribbean,442.0,1981,Antigua and Barbuda,Constitutional Monarchy,Saint John's,-61.845600,17.117500
8,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.370500,24.476400
9,ARG,Argentina,South America,South America,2780400.0,1816,Argentina,Federal Republic,Buenos Aires,-58.417300,-34.611800


In [11]:
sql_query('''SELECT cities.name AS city, countries.country_name AS country, region
            FROM cities
            INNER JOIN countries
            ON cities.country_code = countries.code;''')

,city,country,region
0,Abidjan,Cote d'Ivoire,Western Africa
1,Abu Dhabi,United Arab Emirates,Middle East
2,Abuja,Nigeria,Western Africa
3,Accra,Ghana,Western Africa
4,Addis Ababa,Ethiopia,Eastern Africa
5,Ahmedabad,India,Southern and Central Asia
6,Alexandria,Egypt,Northern Africa
7,Algiers,Algeria,Northern Africa
8,Almaty,Kazakhstan,Southern and Central Asia
9,Ankara,Turkey,Middle East
